In [25]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score, roc_auc_score

In [26]:
validation_games = [
'ESPORTSTMNT02_3041846',
'ESPORTSTMNT02_3041862',
'ESPORTSTMNT02_3041903',
'ESPORTSTMNT02_3041937',
'ESPORTSTMNT02_3041983',
'ESPORTSTMNT02_3041990',
'ESPORTSTMNT02_3042003',
'ESPORTSTMNT02_3042025',
'ESPORTSTMNT02_3042187',
'ESPORTSTMNT02_3042251',
'ESPORTSTMNT02_3052761',
'ESPORTSTMNT02_3052797',
'ESPORTSTMNT02_3042301',
'ESPORTSTMNT02_3052843',
'ESPORTSTMNT02_3052853',
'ESPORTSTMNT02_3052865',
'ESPORTSTMNT02_3053150',
'ESPORTSTMNT02_3053187',
'ESPORTSTMNT02_3053204',
'ESPORTSTMNT02_3053238',
'ESPORTSTMNT02_3053254',
'ESPORTSTMNT02_3053274',
'ESPORTSTMNT02_3053295',
'ESPORTSTMNT02_3053311',
'ESPORTSTMNT02_3042846',
'ESPORTSTMNT02_3042864',
'ESPORTSTMNT02_3042881',
'ESPORTSTMNT02_3042903',
'ESPORTSTMNT02_3042921',
'ESPORTSTMNT02_3042943',
'ESPORTSTMNT02_3042965',
'ESPORTSTMNT02_3042987',
'ESPORTSTMNT02_3043190',
'ESPORTSTMNT02_3043245',
'ESPORTSTMNT02_3043269',
'ESPORTSTMNT02_3043280',
'ESPORTSTMNT02_3043302',
'ESPORTSTMNT02_3043333',
'ESPORTSTMNT02_3043345',
'ESPORTSTMNT02_3043363',
'ESPORTSTMNT02_3043656',
'ESPORTSTMNT02_3043674',
'ESPORTSTMNT02_3043685',
'ESPORTSTMNT02_3043709',
'ESPORTSTMNT02_3043731',
'ESPORTSTMNT02_3043753',
'ESPORTSTMNT02_3043758',
'ESPORTSTMNT02_3045120',
'ESPORTSTMNT02_3045181',
'ESPORTSTMNT02_3045196',
'ESPORTSTMNT02_3045209',
'ESPORTSTMNT02_3045232',
'ESPORTSTMNT02_3045259',
'ESPORTSTMNT02_3045416',
'ESPORTSTMNT02_3045459',
'ESPORTSTMNT02_3045472',
'ESPORTSTMNT02_3045480',
'ESPORTSTMNT02_3045503',
'ESPORTSTMNT02_3045518',
'ESPORTSTMNT02_3045836',
'ESPORTSTMNT02_3045853',
'ESPORTSTMNT02_3045864',
'ESPORTSTMNT02_3045877',
'ESPORTSTMNT02_3056381',
'ESPORTSTMNT02_3056394',
'ESPORTSTMNT02_3056626',
'ESPORTSTMNT02_3056652',
'ESPORTSTMNT02_3056673',
'ESPORTSTMNT02_3056681',
'ESPORTSTMNT02_3056705',
'ESPORTSTMNT02_3056721',
'ESPORTSTMNT02_3057453',
'ESPORTSTMNT02_3057478',
'ESPORTSTMNT02_3057490',
'ESPORTSTMNT02_3057502',
'ESPORTSTMNT02_3057505',
'ESPORTSTMNT02_3057516',
'ESPORTSTMNT02_3057653',
'ESPORTSTMNT02_3057673',
'ESPORTSTMNT02_3057686',
'ESPORTSTMNT02_3057715',
'ESPORTSTMNT02_3057724',
'ESPORTSTMNT02_3057747',
'ESPORTSTMNT02_3047499',
'ESPORTSTMNT02_3057859',
'ESPORTSTMNT02_3057882',
'ESPORTSTMNT02_3057894',
'ESPORTSTMNT02_3057911',
'ESPORTSTMNT02_3057916',
'ESPORTSTMNT02_3057923',
'ESPORTSTMNT02_3057927',
'ESPORTSTMNT02_3047802',
'ESPORTSTMNT02_3047839',
'ESPORTSTMNT02_3047847',
'ESPORTSTMNT02_3047861',
'ESPORTSTMNT02_3047878',
'ESPORTSTMNT02_3047898',
'ESPORTSTMNT02_3047918',
'ESPORTSTMNT02_3048340',
'ESPORTSTMNT02_3048354',
'ESPORTSTMNT02_3048364',
'ESPORTSTMNT02_3048468',
'ESPORTSTMNT02_3048492',
'ESPORTSTMNT02_3048494',
'ESPORTSTMNT02_3048616',
'ESPORTSTMNT02_3048629',
'ESPORTSTMNT02_3048640',
'ESPORTSTMNT02_3048649',
'ESPORTSTMNT02_3048651',
'ESPORTSTMNT02_3048763',
'ESPORTSTMNT02_3048774',
'ESPORTSTMNT02_3048779',
'ESPORTSTMNT02_3048784',
'ESPORTSTMNT02_3048787',
'ESPORTSTMNT02_3059738',
'ESPORTSTMNT02_3059755',
'ESPORTSTMNT02_3059756',
'ESPORTSTMNT02_3059758',
'ESPORTSTMNT02_3059851',
'ESPORTSTMNT02_3059852',
'ESPORTSTMNT02_3059853',
'ESPORTSTMNT02_3059854'
]

In [27]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
validation_df = df.apply(lambda row: row[df['game'].isin(validation_games)])
df = df[~df.game.isin(validation_games)]
df

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result
0,36864,ESPORTSTMNT02_2556988,19,1,2,1,3,24,2,23,...,14,0.71,4.2,1,0.00,2.5,1,1.00,7.0,1
1,36865,ESPORTSTMNT05_2520933,19,21,1,22,25,24,20,21,...,8,0.38,3.2,21,0.57,3.3,6,0.83,4.1,0
2,36866,ESPORTSTMNT02_2557390,19,2,20,24,2,4,20,3,...,5,0.80,4.8,1,0.00,0.8,5,0.60,3.2,0
3,36867,ESPORTSTMNT02_2557426,20,0,21,20,21,5,23,1,...,15,0.67,3.9,34,0.74,7.3,2,0.00,2.0,0
4,36868,ESPORTSTMNT02_2557463,19,1,2,1,2,4,20,24,...,5,0.40,2.4,2,1.00,24.0,1,1.00,6.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4584,44417,ESPORTSTMNT02_3080864,19,1,21,1,2,4,1,5,...,29,0.69,5.8,55,0.71,5.6,7,0.57,2.1,1
4585,44418,ESPORTSTMNT02_3080870,21,19,20,2,22,1,4,3,...,39,0.77,4.6,5,0.60,6.6,7,0.86,6.3,1
4586,44419,ESPORTSTMNT02_3080871,0,20,21,5,1,21,22,1,...,30,0.67,5.4,87,0.64,4.7,11,0.55,2.8,1
4587,44420,ESPORTSTMNT02_3080872,19,20,21,1,2,5,4,3,...,27,0.67,4.3,14,0.57,3.5,1,1.00,20.0,1


In [28]:
y = df['result'].copy()
X = df.drop(['golId','result','game','event4','event5','event6','event7','event8','event9','event10','event11','event12','event13','event14','event15','event16','event17','event18','event19','event20','event21','event22','event23','event24','event25','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
X

,event1,event2,event3,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,...,redJungleKDA,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA
0,19,1,2,1,1.00,2.0,1,1.00,2.3,1,...,0.0,14,0.71,4.2,1,0.00,2.5,1,1.00,7.0
1,19,21,1,5,0.60,5.6,2,0.00,1.1,9,...,3.5,8,0.38,3.2,21,0.57,3.3,6,0.83,4.1
2,19,2,20,0,0.00,0.0,1,1.00,3.3,0,...,0.0,5,0.80,4.8,1,0.00,0.8,5,0.60,3.2
3,20,0,21,23,0.61,3.6,5,0.20,1.6,8,...,4.2,15,0.67,3.9,34,0.74,7.3,2,0.00,2.0
4,19,1,2,3,1.00,9.8,2,1.00,7.0,1,...,4.7,5,0.40,2.4,2,1.00,24.0,1,1.00,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4584,19,1,21,5,1.00,4.8,42,0.88,5.4,113,...,3.8,29,0.69,5.8,55,0.71,5.6,7,0.57,2.1
4585,21,19,20,21,0.52,2.5,22,0.59,3.6,30,...,3.8,39,0.77,4.6,5,0.60,6.6,7,0.86,6.3
4586,0,20,21,11,1.00,9.5,8,0.63,3.8,114,...,3.6,30,0.67,5.4,87,0.64,4.7,11,0.55,2.8
4587,19,20,21,34,0.47,2.9,1,1.00,13.0,22,...,13.3,27,0.67,4.3,14,0.57,3.5,1,1.00,20.0


In [29]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(random_state=42,max_iter=50000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(random_state=42,max_iter=50000),
    'Support Vector Machine (RBF Kernel)': SVC(random_state=42,C=100,gamma=0.001,kernel='rbf',max_iter=50000),
    'Decission Tree': DecisionTreeClassifier(random_state=42),
    'Adaboost': AdaBoostClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42,max_depth=25,n_estimators=600,min_samples_split=2,min_samples_leaf=1),
    'Gradient Boosting Classifier': GradientBoostingClassifier(random_state=42,criterion='friedman_mse',learning_rate=0.025,loss='deviance',max_depth=4,max_features='log2',min_samples_leaf=8,min_samples_split=3, n_estimators=100,subsample=0.5)
}


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained
Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [31]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Balanced Accuracy': balanced_accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test)),
    'ROC/AUC Score': roc_auc_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [32]:
scores

,Model,Balanced Accuracy,Precision,Recall,F1-Score,ROC/AUC Score
0,Logistic Regression,0.640501,0.642105,0.671806,0.656620,0.640501
1,Support Vector Machine (Linear Kernel),0.643757,0.643750,0.680617,0.661670,0.643757
2,Support Vector Machine (RBF Kernel),0.642655,0.643006,0.678414,0.660236,0.642655
3,Decission Tree,0.563692,0.575000,0.557269,0.565996,0.563692
4,Adaboost,0.635951,0.637317,0.669604,0.653061,0.635951
5,Random Forest,0.639544,0.643162,0.662996,0.652928,0.639544
6,Gradient Boosting Classifier,0.636336,0.642082,0.651982,0.646995,0.636336


In [33]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))

In [34]:
y_validation = validation_df['result'].copy()
X_validation = validation_df.drop(['golId','result','game','event4','event5','event6','event7','event8','event9','event10','event11','event12','event13','event14','event15','event16','event17','event18','event19','event20','event21','event22','event23','event24','event25','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
X_validation

,event1,event2,event3,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,...,redJungleKDA,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA
4010,0,21,20,31,0.55,3.1,22,0.55,4.3,0,...,7.6,10,0.50,4.5,25,0.56,4.6,7,0.57,2.4
4011,0,21,20,31,0.55,3.1,22,0.55,4.3,0,...,7.6,10,0.50,4.5,25,0.56,4.6,7,0.57,2.4
4012,20,19,2,27,0.41,2.6,0,0.00,0.0,18,...,4.9,45,0.62,5.5,0,0.00,0.0,94,0.60,4.1
4013,19,21,20,25,0.56,2.5,30,0.60,3.3,33,...,4.5,10,0.70,8.3,17,0.82,6.2,10,0.60,3.3
4014,19,21,23,32,0.50,2.7,0,0.00,0.0,9,...,5.5,10,0.40,6.5,0,0.00,0.0,4,1.00,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4579,19,2,4,17,0.65,4.5,20,0.60,3.0,18,...,5.2,39,0.77,6.0,70,0.59,3.9,20,0.35,3.6
4580,19,2,4,17,0.65,4.5,20,0.60,3.0,18,...,5.2,39,0.77,6.0,70,0.59,3.9,20,0.35,3.6
4581,19,1,2,6,0.83,6.3,37,0.49,3.7,49,...,3.5,23,0.70,6.8,91,0.60,5.0,10,0.50,2.4
4582,19,1,2,6,0.83,6.3,37,0.49,3.7,49,...,3.5,23,0.70,6.8,91,0.60,5.0,10,0.50,2.4


In [35]:
val_scores_list = []

for name,model in models.items():    
    val_scores_list.append({
    'Model': name,
    'Balanced Accuracy': balanced_accuracy_score(y_validation,model.predict(X_validation)),
    'Precision':  precision_score(y_validation,model.predict(X_validation)),
    'Recall': recall_score(y_validation,model.predict(X_validation)),
    'F1-Score': f1_score(y_validation,model.predict(X_validation)),
    'ROC/AUC Score': roc_auc_score(y_validation,model.predict(X_validation))
    })
val_scores = pd.DataFrame(val_scores_list)

c:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
val_scores

,Model,Balanced Accuracy,Precision,Recall,F1-Score,ROC/AUC Score
0,Logistic Regression,0.580651,0.782609,0.233766,0.360000,0.580651
1,Support Vector Machine (Linear Kernel),0.574158,0.772727,0.220779,0.343434,0.574158
2,Support Vector Machine (RBF Kernel),0.505741,0.666667,0.025974,0.050000,0.505741
3,Decission Tree,0.494542,0.524194,0.844156,0.646766,0.494542
4,Adaboost,0.591756,0.592233,0.792208,0.677778,0.591756
5,Random Forest,0.519669,0.564103,0.285714,0.379310,0.519669
6,Gradient Boosting Classifier,0.500000,0.000000,0.000000,0.000000,0.500000
